In [1]:
import torch
import nnsight
from PIL import Image
import os, json, argparse, itertools, math, torch, numpy as np
from tqdm import tqdm
from transformers import AutoProcessor
from vllm.inputs import TextPrompt
from nnsight.modeling.vllm import VLLM
import einops
import json

torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [2]:
print(model)

LlavaForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-22): 23 x CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (qkv_proj): QKVParallelLinear(in_features=1024, output_features=3072, bias=True, tp_size=1, gather_output=False)
              (out_proj): RowParallelLinear(input_features=1024, output_features=1024, bias=True, tp_size=1, reduce_results=True)
              (attn): MultiHeadAttention()
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): QuickGELU()
           

In [4]:

index = 1
clean_image = Image.open(f'./scripts/dataset/clean-{index}.jpg')
corrupted_image = Image.open(f'./scripts/dataset/corrupted-{index}.jpg')
prompt = " <image> The color of an apple is[/INST]"
clean_index = torch.tensor(model.tokenizer("button").input_ids)[1].max().item()
corrupted_index = torch.tensor(model.tokenizer("tree").input_ids)[1].max().item()

print(clean_index)
print(corrupted_index)
NUM_LAYERS = len(model.language_model.model.layers)
NUM_HEADS = int(model.language_model.model.config.num_attention_heads)


2826
5447


In [6]:
with model.trace(temperature=0.0, top_p=1.0) as tracer:
  with tracer.invoke(TextPrompt(prompt=prompt,multi_modal_data={"image": clean_image})) as result:
    model.language_model.model.layers[31].output[0][-1] = model.language_model.model.layers[31].output[0][-3]
    model.language_model.model.layers[31].output[1][-1] = model.language_model.model.layers[31].output[1][-3]
    logits = model.logits.output.save()

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 14.97it/s, est. speed input: 8841.02 toks/s, output: 90.02 toks/s]


In [7]:

i = logits.argmax().item()
print(model.tokenizer.decode(i))


NNsightError: Accessing value before it's been set.

In [2]:
vllm = VLLM("gpt2", device = "auto", dispatch = True)

INFO 05-04 13:20:06 config.py:1861] Downcasting torch.float32 to torch.float16.
INFO 05-04 13:20:10 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='gpt2', speculative_config=None, tokenizer='gpt2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=gpt2, num_scheduler_steps=1, chunked_prefill_enabled=False multi_step_stream_outputs=True, enable_prefix_caching=Fa

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 05-04 13:20:11 model_runner.py:1077] Loading model weights took 0.2378 GB
INFO 05-04 13:20:12 worker.py:232] Memory profiling results: total_gpu_memory=139.72GiB initial_memory_usage=0.85GiB peak_torch_memory=0.73GiB memory_usage_post_profile=0.95GiB non_torch_memory=0.68GiB kv_cache_size=124.34GiB gpu_memory_utilization=0.90
INFO 05-04 13:20:12 gpu_executor.py:113] # GPU blocks: 226353, # CPU blocks: 7281
INFO 05-04 13:20:12 gpu_executor.py:117] Maximum concurrency for 1024 tokens per request: 3536.77x
INFO 05-04 13:20:14 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-04 13:20:14 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.


In [3]:
print(vllm)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): VocabParallelEmbedding(num_embeddings=50304, embedding_dim=768, org_vocab_size=50257, num_embeddings_padded=50304, tp_size=1)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): QKVParallelLinear(in_features=768, output_features=2304, bias=True, tp_size=1, gather_output=False)
          (c_proj): RowParallelLinear(input_features=768, output_features=768, bias=True, tp_size=1, reduce_results=True)
          (attn): Attention(head_size=64, num_heads=12, num_kv_heads=12, scale=0.125, backend=FlashAttentionImpl)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): ColumnParallelLinear(in_features=768, output_features=3072, bias=True, tp_size=1, gather_output=False)
          (c_proj): RowParallelLinear(input_features=

In [4]:
with vllm.trace(temperature=0.0, top_p=1.0, max_tokens=1) as tracer:
  with tracer.invoke("The Eiffel Tower is located in the city of"):
    clean_logits = vllm.logits.output.save()

  with tracer.invoke("The Eiffel Tower is located in the city of"):
    vllm.transformer.h[-2].mlp.output[:] = 0
    corrupted_logits = vllm.logits.output.save()


The history saving thread hit an unexpected error (IndexError('list index out of range')).History will not be written to the database.


IOStream.flush timed out
Processed prompts: 100%|██████████| 2/2 [00:00<00:00, 219.45it/s, est. speed input: 2436.81 toks/s, output: 221.36 toks/s]


In [4]:
with vllm.trace(temperature=0.0, top_p=1.0, max_tokens=1) as tracer:
  with tracer.invoke("The Eiffel Tower is located in the city of"):
    clean_logits = vllm.logits.output.save()

  with tracer.invoke("The Eiffel Tower is located in the city of"):
    vllm.transformer.h[-2].mlp.output[:] = 0
    corrupted_logits = vllm.logits.output.save()


Processed prompts: 100%|██████████| 2/2 [00:00<00:00, 122.40it/s, est. speed input: 1352.92 toks/s, output: 122.94 toks/s]


In [5]:
print("\nCLEAN - The Eiffel Tower is located in the city of", vllm.tokenizer.decode(clean_logits.argmax(dim=-1)))
print("\nCORRUPTED - The Eiffel Tower is located in the city of", vllm.tokenizer.decode(corrupted_logits.argmax(dim=-1)))



CLEAN - The Eiffel Tower is located in the city of  Paris

CORRUPTED - The Eiffel Tower is located in the city of  London
